In [ ]:
%matplotlib inline

import os
import subprocess

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter


def measure_us(core: int, num_page: int) -> int:
  # Define the command you want to run
  command = ["taskset", "-c", str(core), "build/tlb", str(num_page), str(num_trial)]
  try:
    # Run the command and capture the output
    result = subprocess.run(command, stdout=subprocess.PIPE)
    return int(result.stdout)
  except subprocess.CalledProcessError as e:
    print("An error occurred while running the command: ", command)
    print(e.stderr)
    os.exit(1)

num_trial = 1000
xmax = 8
ymax = 0
num_cores = os.cpu_count()

num_pages = [2 ** i for i in range(xmax + 1)]
for end_pow in range(3, xmax + 1):
  end = 2 ** end_pow
  start, step = end // 2, end // 8
  num_pages.extend(range(start, end, step))
num_pages = sorted(set(num_pages))

# Create the plot
plt.figure(figsize=(8, 8))
plt.title('TLB Size Measurement')
plt.xlabel('Number Of Pages')
plt.ylabel('Time Per Access (ns)')
plt.axhline(0, color='black',linewidth=0.5, ls='--')  # Add x-axis
plt.axvline(0, color='black',linewidth=0.5, ls='--')  # Add y-axis
plt.grid(color = 'gray', linestyle = '--', linewidth = 0.5)  # Add grid
plt.xlim(1, xmax)
plt.xscale('log', base=2)
ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(1)) # Place tick marks at every multiple of 1, i.e. at any integer
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, pos: int(x))) # Format the tick label to be 2 raised to the power of `x`
plt.gcf().autofmt_xdate() # Make the axis labels rotated for easier reading
plt.xticks([2 ** i for i in range(xmax)])

colors = ['orange', 'blue', 'green', 'red']
markers = "o^s*"
num_cores = min(num_cores, 4)
for core in range(num_cores):
  color, marker = colors[core], markers[core]
  times_ns = []
  for num_page in num_pages:
    runs = 10
    sum_time_us = sum([measure_us(core, num_page) for _ in range(runs)]) // runs
    times_ns.append(int(sum_time_us * 1000 // (num_page * num_trial)))

  x, y = num_pages, times_ns
  ymax = max(ymax, max(y))
  plt.plot(x, y, label=f'CPU #{core}', color=color, marker=marker)  # Plot the line

plt.ylim(0, ymax + 1)  # Set y-axis limits
plt.legend()

plt.show()